# Build MultiMNIST_index_test/train.npf 

In [0]:
!fusermount -u drive
!rm -r *
#!find . ! -name 'drive' -type d -exec rm -d {} +
!dir
!free -g
#!cat /proc/meminfo

In [0]:
import tensorflow as tf
import numpy as np
from pdb import set_trace

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def make_tf_example(image_string, classes):
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                'image': tf.train.Feature(
                    bytes_list=tf.train.BytesList(value=[image_string])),
                'label': tf.train.Feature(
                    int64_list=tf.train.Int64List(value=classes)),
            }
        )
    )


def pad_to44(img, shape=[44, 44, 1]):
    '''shape=[h, w]'''
    result = np.zeros([44, 44, 1])
    result[8:36, 8:36, :] = img
    return result


def random_crop(img, shape=36):
    i, j = np.random.choice(range(9), 2)
    return img[i: i + shape, j: j + shape, :]


class MultiMNISTBuilder(object):
    def __init__(self, n_proliferation=1000, num_class=10, shape=[36, 36, 1]):
        self.num_class = num_class
        self.n_per_class, self.remainder = divmod(
            n_proliferation, num_class - 1)

        # a simple TF graph here
        self.np_img = tf.placeholder(tf.uint8, shanp.pe=shape)
        self.png_img = tf.image.encode_png(self.np_img)
        
        self.sess = None
        self.tfr_writer = None

    
    def build(self, oFilename, target='training'):
        ''' build training or testing set '''
        if target == 'training':
            x, _ = self._load_mnist()
            N_OUTPUT = 60000000
        elif target == 'testing':
            _, x = self._load_mnist()
            N_OUTPUT = 10000000
        else:
            raise ValueError('Only `training` and `testing` are supported.')

        c = 1
        self.sess = tf.Session()
        tfr_writer = tf.python_io.TFRecordWriter(oFilename)
        for i in range(10):
            x_digit_i = x[i]
            other_class = set(range(10)) - set([i])
            for xi in x_digit_i:
                xi = random_crop(pad_to44(xi)) 
                for j in other_class:
                    Nj = x[j].shape[0]

                    index = np.random.choice(range(Nj), self.n_per_class, replace=False)
                    imgs_from_that_class = x[j][index]

                    for xo in imgs_from_that_class:
                        self._pad_crop_merge_save((xi, i), (xo, j), tfr_writer)
                        print('\rProcessing {:08d}/{:08d}...'.format(c, N_OUTPUT), end='')
                        c += 1

                for _ in range(self.remainder):
                    j = np.random.choice(list(other_class))
                    Nj = x[j].shape[0]
                    index = np.random.choice(range(Nj))
                    xo = x[j][index]

                    self._pad_crop_merge_save((xi, i), (xo, j), tfr_writer)
                    print('\rProcessing {:08d}/{:08d}...'.format(c, N_OUTPUT), end='')
                    c += 1
        print()
        self.sess.close()
        tfr_writer.close()

    def _load_mnist(self):
        (x, y), (x_t, y_t) = tf.keras.datasets.mnist.load_data()

        if len(x.shape) == 3:
            x = np.expand_dims(x, -1)
            x_t = np.expand_dims(x_t, -1)

        x = [x[y==i] for i in range(self.num_class)]
        x_t = [x_t[y_t==i] for i in range(self.num_class)]
        return x, x_t

    def _pad_crop_merge_save(self, xi_i, xo_j, writer):
        xi, i = xi_i
        xo, j = xo_j
        xo = random_crop(pad_to44(xo))
        combined_img = np.concatenate([xi, xo], -1)
        combined_img = np.max(combined_img, -1, keepdims=True)

        png_encoded = self.sess.run(
            self.png_img, feed_dict={self.np_img: combined_img})

        ex = make_tf_example(png_encoded, [i, j])
        writer.write(ex.SerializeToString())

class MultiMNISTIBuilder2(MultiMNISTBuilder):
    def __init__(self, n_proliferation=1000, num_class=10, shape=[36, 36, 1]):
        self.num_class = num_class
        self.n_per_class, self.remainder = divmod(
            n_proliferation, num_class - 1)

        self.n_proliferation = n_proliferation
        # a simple TF graph here
        self.np_img = tf.placeholder(tf.uint8, shape=shape)
        self.png_img = tf.image.encode_png(self.np_img)
        
        self.sess = None
        self.tfr_writer = None

    def build(self, oFilename, target='training'):
        # ''' build training or testing set '''
        # if target == 'training':
        #     x, _ = self._load_mnist()
        #     N_OUTPUT = 60000000
        #     N = 60000
        # elif target == 'testing':
        #     _, x = self._load_mnist()
        #     N_OUTPUT = 10000000
        #     N = 10000
        # else:
        #     raise ValueError('Only `training` and `testing` are supported.')

        (x, y), (x_t, y_t) = tf.keras.datasets.mnist.load_data()
        N_train = x.shape[0]
        N_test = x_t.shape[0]
        N_proliferate = 1000
        K = 10
        H = 8  # allowable starting index (from top-left)
        W = 8  # allowable starting index (from top-left)

        # for training
        # def (x, y, oFilename, shapes=[K, N, M, P])
        N = N_train
        M = N_proliferate
        P = 3  # index, h_i, w_i

        output = np.zeros([N, M, P], dtype=int)
        index_all = set(range(N))
        index_of_class = set([np.where(y==i)[0] for i in range(K)])
        for k in range(K):
            index_of_other_class = list(index_all - index_of_class[k])
            for i in index_of_class[k]:
                output[i, :, 0] = np.random.choice(
                    index_of_other_class, M, replace=False)
                output[i, :, 1] = np.random.choice(range(H + 1), M)
                output[i, :, 2] = np.random.choice(range(W + 1), M)

        with open('MultiMNIST_index_train.npf', wb) as fp:
            fp.write(output.tostring())

class MultiMNISTIndexBuilder(object):
    def __init__(self, num_class=10, num_proliferate=1000, H=8, W=8):
        ''' # allowable starting index (from top-left)'''
        (x, y), (x_t, y_t) = tf.keras.datasets.mnist.load_data()
        self.num_class = num_class
        self.num_proliferate = num_proliferate

        self.H = H
        self.W = W

        self._build(x, y, 'MultiMNIST_index_train.npf')
        self._build(x_t, y_t, 'MultiMNIST_index_test.npf')
    
    def _get_kmp(self):
        return self.num_class, self.num_proliferate, 3

    def _get_hw(self):
        return self.H, self.W

    def _build(self, x, y, oFilename):
        K, M, P = self._get_kmp()
        N = x.shape[0]
        H, W = self._get_hw()

        output = np.zeros([N, M, P], dtype=np.int64)
        index_all = set(range(N))
        index_of_class = [np.where(y==i)[0] for i in range(K)]
        counter = 1
        for k in range(K):
            index_of_other_class = list(index_all - set(index_of_class[k]))
            for i in index_of_class[k]:
                print('\rProcessing {:5d}/{:5d}'.format(counter, N), end='')
                counter += 1
                # set_trace()
                output[i, :, 0] = np.random.choice(
                    index_of_other_class, M, replace=False)
                output[i, :, 1] = np.random.choice(range(H + 1), M)
                output[i, :, 2] = np.random.choice(range(W + 1), M)
            index_all = set(range(N))
        print()

        with open(oFilename, 'wb') as fp:
            fp.write(output.tostring())


if __name__ == '__main__':
    # builder = MultiMNISTBuilder()
    # builder.build('./MultiMNIST_train.tfr', 'training')
    # builder.build('./MultiMNIST_test.tfr', 'testing')  # about 2 hr
    builder = MultiMNISTIndexBuilder()

In [0]:
!dir

# Git

In [0]:
!git clone https://github.com/JeremyCCHsu/CapsNet-tf

In [0]:
# !git clone https://github.com/IronChariot/cnn-multi-mnist
# import os
# os.chdir('base_mnist')

In [0]:
!mv MultiMNIST_index_train.npf CapsNet-tf
!mv MultiMNIST_index_test.npf CapsNet-tf

In [0]:
import os
os.chdir('CapsNet-tf')

In [0]:
#!mkdir multiMNIST

In [0]:
# import os
# os.chdir('..')

In [0]:
# !mv multiMNIST CapsNet-tf

In [0]:
!dir CapsNet-tf

In [0]:
# !cat helper.py

In [0]:
# !python multi_mnist_setup.py

# Atiro nem kell!!!!!!!!!

In [0]:
%%writefile  train_multimnist.py
import json

import numpy as np
import tensorflow as tf

from helper import *
from capsule import CapsuleMultiMNIST

args = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string(
    'arch', 'architecture-multimnist.json', 'network architecture')
tf.app.flags.DEFINE_string(
    'logdir_root', None, 'root of log dir')
tf.app.flags.DEFINE_string('logdir', None, 'log dir')
tf.app.flags.DEFINE_string(
    'restore_from', None, 'restore from dir (not from *.ckpt)')
tf.app.flags.DEFINE_string('msg', '-MultiMNIST', 'Additional message')


def main():
    with open(args.arch) as fp:
        arch = json.load(fp)
    data = MultiMNISTIndexReader(
        train_index='MultiMNIST_index_train.npf',
        batch_size=arch['training']['batch_size'],
        data_format='channels_last',
        capacity=2**10, min_after_dequeue=2**9
    )
    sess = tf.Session()
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    with sess.as_default():
       juj=data.x.eval()
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print(type(data))
    dirs = validate_log_dirs(args)
    arch.update({'logdir': dirs['logdir']})
    net = CapsuleMultiMNIST(arch=arch)
    loss = net.loss(data.x, data.y, data.xi, data.xj)
    # net.inspect(data.example)
    # loss_t = net.loss(data.x_t, data.y_t)
    net.train(loss, loss_t=None)
#     oFilename=
    with open(juj.txt, 'wb') as fp:
             fp.write(juj)
if __name__ == '__main__':
    main()
    


In [0]:
!python build_multimnist.py

In [0]:
!python train_multimnist.py

# Kinyero MultiMNIST_index_test.npf 

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

import operator as op
from functools import reduce
def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer//denom

def char2tanh(x):
    ''' [0, 255] -> [-1., 1.] '''
    return x / 127.5 - 1.
def multi(
        train_index='MultiMNIST_index_test.npf',  # TODO
        batch_size=32,
        batch_size_t=100, 
        capacity=512,
        min_after_dequeue=256,
        # shift=None,  # TODO
        # dimension=28,
        seed_=9527,
        meret=10,
        num_threads=4,
        ):
        with tf.device('cpu'):
            with tf.name_scope('MNISTInputPipeline'):   
              np.random.seed(seed_)
              tf.set_random_seed(seed_)
              (x, y), (x_t, y_t) = tf.keras.datasets.mnist.load_data()
              y, y_t = tf.cast(y, tf.int64), tf.cast(y_t, tf.int64)        
              # x, x_t = char2tanh(x), char2tanh(x_t)
              
              x, x_t = tf.constant(x, dtype=tf.float32, name='x'), tf.constant(x_t, dtype=tf.float32)
              x, x_t = tf.expand_dims(x, -1), tf.expand_dims(x_t, -1)
              # exemplar data
              example = tf.gather(x, [1, 3, 5, 7, 2, 0, 13, 15, 17, 4])

              # Training
              i = np.fromfile(train_index, np.int64)
              i = np.reshape(i, [x.shape[0], -1, 3])
              i = tf.constant(i[:, :, 0])  # [N=60K, M=1K]
              
              x_i, y_i, i_i = tf.train.slice_input_producer([x, y, i],seed=seed_)  # i_i: [M=1K,]
              ii = tf.random_uniform([], 0, 1000, tf.int64,seed=seed_)
              ii = i_i[ii] # []

              x_j = x[ii]  # [28, 28, 1]
              y_j = y[ii]  # []
              y_j = tf.reshape(y_j, [1,])
              y_i = tf.reshape(y_i, [1,])

              x_i = tf.image.resize_image_with_crop_or_pad(x_i, 44, 44)
              x_i = tf.random_crop(x_i, size=[36, 36, 1],seed=seed_)
              x_i = char2tanh(x_i)

              x_j = tf.image.resize_image_with_crop_or_pad(x_j, 44, 44)
              x_j = tf.random_crop(x_j, size=[36, 36, 1], seed=seed_)  # TODO
              x_j = char2tanh(x_j)

              x_merge = tf.concat(
                  [
                      tf.expand_dims(x_i, -1),
                      tf.expand_dims(x_j, -1)
                  ], 
                  -1
              )
              x_merge = tf.reduce_max(x_merge, -1) # [b=1, h, w, c]
              y_merge = tf.concat([y_i, y_j], -1)
              print(y_merge[0,...])
              x, y, xi,xj = tf.train.shuffle_batch(
                  [x_merge, y_merge, x_i, x_j],
                  batch_size=batch_size,
                  capacity=capacity,
                  seed=9527,
                  min_after_dequeue=min_after_dequeue,
                  # enqueue_many=True,
                  num_threads=num_threads,
              )
#               xi=x_i
#               xj=x_j
#               x=x_merge
#               y=y_merge
              print('YYYYYYYXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXxx')
              with tf.Session() as sess:
                  tf.train.start_queue_runners(sess) 
                  sample_images,y_,eredeti1,eredeti2=sess.run([x,y,xi,xj])

              print('zzzzzzXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXxx')
              #print(y_)
              #input("Press Enter to continue...")
              eredeti1 = np.asarray(eredeti1).reshape(-1, 36, 36)
              eredeti2 = np.asarray(eredeti2).reshape(-1, 36, 36)
              sample_images = np.asarray(sample_images).reshape(-1, 36, 36)
              n_samples=batch_size
              
              plt.figure(figsize=(3.2, (meret+1)*1.3),facecolor='g')
              labels=[]#np.empty((0, batch_size))
              for index in range(n_samples):
                  if y_[index][0]>y_[index][1]:
                    a=y_[index][0]
                    y_[index][0]=y_[index][1]
                    y_[index][1]=a
                    #eredeti2[index],eredeti1[index]=eredeti1[index],eredeti2[index]
                    b=np.ones(shape=[36,36])
                    b=np.copy(eredeti2[index][:])
                    eredeti2[index]=np.copy(eredeti1[index][:])
                    eredeti1[index]=np.copy(b[:])
                  szam=str(y_[index][0]*10+y_[index][1])
                  #label= y_[index][0]*10+y_[index][1]-ncr(y_[index][0]+2,2)
                  label=y_[index]
                  #print(labels.shape)
#                   print(np.array(label).reshape([-1,1]).shape)
#                   print([np.array(label).reshape([1])].shape)
                  
                  labels.extend([label])#np.array(label).reshape([1])
                  new_p = (sample_images[index] * 255).astype(np.uint8)
                  new_p = Image.fromarray(new_p)
                  new_p1 = (eredeti1[index] * 255).astype(np.uint8)
                  new_p1 = Image.fromarray(new_p1)
                  new_p2 = (eredeti2[index] * 255).astype(np.uint8)
                  new_p2 = Image.fromarray(new_p2)
                  
                  if new_p.mode != 'RGB':
                    new_p = new_p.convert('RGB')
                  
                  print("SAVEEEEEEEEEEEEEEEEEEEEEEEE")
                  
                  
                  
                  #egy mappaba
                  new_p.save("../multi/{}.bmp".format(index))
                  new_p1.save("../original1/{}_1.bmp".format(index))
                  new_p2.save("../original2/{}_2.bmp".format(index))
                  
                  #kulon mappaba
#                   new_p.save("./multiMNIST/multi/{}/{}.bmp".format(szam,index))
#                   new_p1.save("./multiMNIST/original1/{}/{}.bmp".format(szam,index))
#                   new_p2.save("./multiMNIST/original2/{}/{}.bmp".format(szam,index))
                  if index<(meret):
                    
#                     plt.subplot(3, 3, (index*3)+1)
#                     plt.imshow(sample_images[index], cmap="binary")
#                     plt.axis("off")
#                     plt.subplot(3, 3, (index*3)+2)
#                     plt.imshow(eredeti1[index], cmap="binary")
#                     plt.axis("off")
#                     plt.title("Címke:" + str(y_[index]))
#                     plt.subplot(3, 3, (index*3)+3)
#                     plt.imshow(eredeti2[index], cmap="binary")
#                     plt.axis("off")
                    
                    plt.subplot(meret, 3, (index*3)+1)
                    plt.imshow(new_p, cmap="binary")
                    plt.axis("off")
                    plt.subplot(meret, 3, (index*3)+2)
                    plt.imshow(eredeti1[index], cmap="binary")
                    plt.axis("off")
                    plt.title("Címke:" + str(y_[index]))
                    plt.subplot(meret, 3, (index*3)+3)
                    plt.imshow(eredeti2[index], cmap="binary")
                    plt.axis("off")
                  
                                
              #plt.show()
              np.array(labels).dump("../labels.pkl")
              print("most")
              print((labels[9999]))
              print((labels[0]))
              
#               return x_merge, y#, xi,xj

In [0]:
import os
os.chdir('..')
!mkdir multi
!mkdir original1
!mkdir original2
os.chdir('CapsNet-tf')

In [0]:
#!mv ./CapsNet-tf/multiMNIST/labels.pkl ./

In [0]:
!dir

In [0]:
import os
for i in range (0,10):
  for m in range (0,10):
    if i==m:
      continue
    else:
      if i>m:
        continue
      else:
        szam=i*10+m
        os.mkdir("./multiMNIST/multi/{}".format(szam))

In [0]:
import os
for i in range (0,10):
  for m in range (0,10):
    if i==m:
      continue
    else:
      if i>m:
        continue
      else:
        szam=i*10+m
        os.mkdir("./multiMNIST/original1/{}".format(szam))

In [0]:
import os
for i in range (0,10):
  for m in range (0,10):
    if i==m:
      continue
    else:
      if i>m:
        continue
      else:
        szam=i*10+m
        os.mkdir("./multiMNIST/original2/{}".format(szam))

In [0]:
multi(
        train_index='MultiMNIST_index_train.npf',
        batch_size=100000,
        capacity=2**10, 
        min_after_dequeue=2**9,
        seed_=9527,
        meret=62,
        num_threads=1       
    )
########################testre is trainre is

In [0]:
import pickle
d=np.load("./labels.pkl")

In [0]:
import os
os.chdir('multiMNIST')

In [0]:
!rm dataset.tar

In [0]:
  !dir

In [0]:
import os
os.chdir('..')
!tar -cvf multi.tar ./multi
!tar -cvf original1.tar ./original1
!tar -cvf original2.tar ./original2

In [0]:
# from google.colab import files
# files.download('dataset.tar') 

In [0]:
!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
!cd Dropbox-Uploader
!chmod +x Dropbox-Uploader/dropbox_uploader.sh
!bash Dropbox-Uploader/dropbox_uploader.sh

In [0]:
!echo "VB0JaVQS98AAAAAAAAAAiNFOaA0-uGGb0KruqdvbXU9jcZNC8CsyfsZGkrBtwjx8" > token.txt
!bash Dropbox-Uploader/dropbox_uploader.sh

In [0]:
!bash Dropbox-Uploader/dropbox_uploader.sh upload labels.pkl labels.pkl
!bash Dropbox-Uploader/dropbox_uploader.sh upload original1.tar original1_10000.tar
!bash Dropbox-Uploader/dropbox_uploader.sh upload original2.tar original2_10000.tar
!bash Dropbox-Uploader/dropbox_uploader.sh upload multi.tar multi_10000.tar
!bash Dropbox-Uploader/dropbox_uploader.sh unlink

In [0]:
!rm -r CapsNet-tf/multiMNIST/*

In [0]:
!ls -lh dataset.tar

In [0]:
!dir

In [0]:
!ls -ld */

In [0]:
import tensorflow as tf
with tf.Session() as sess:
  sample_images=sess.run([x1])

# Gergon

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True

#config = tf.ConfigProto(device_count = {'GPU': 0})
#sess = tf.Session(config=config)


tf.reset_default_graph()
np.random.seed(42)
tf.set_random_seed(42)

from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/")

X = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32, name="X")



caps1_n_maps = 32
caps1_n_caps = caps1_n_maps * 6 * 6  # 1152 primary capsules
caps1_n_dims = 8


conv1_params = {
    "filters": 256,
    "kernel_size": 9,
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

conv2_params = {
    "filters": caps1_n_maps * caps1_n_dims,  # 256 convolutional filters
    "kernel_size": 9,
    "strides": 2,
    "padding": "valid",
    "activation": tf.nn.relu
}


conv1 = tf.layers.conv2d(X, name="conv1", **conv1_params)
conv2 = tf.layers.conv2d(conv1, name="conv2", **conv2_params)



caps1_raw = tf.reshape(conv2, [-1, caps1_n_caps, caps1_n_dims],
                       name="caps1_raw")


def squash(s, axis=-1, epsilon=1e-7, name=None):
    with tf.name_scope(name, default_name="squash"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector


caps1_output = squash(caps1_raw, name="caps1_output")

caps2_n_caps = 10
caps2_n_dims = 16


init_sigma = 0.1

W_init = tf.random_normal(
    shape=(1, caps1_n_caps, caps2_n_caps, caps2_n_dims, caps1_n_dims),
    stddev=init_sigma, dtype=tf.float32, name="W_init")
W = tf.Variable(W_init, name="W")

batch_size = tf.shape(X)[0]
W_tiled = tf.tile(W, [batch_size, 1, 1, 1, 1], name="W_tiled")

caps1_output_expanded = tf.expand_dims(caps1_output, -1,
                                       name="caps1_output_expanded")
caps1_output_tile = tf.expand_dims(caps1_output_expanded, 2,
                                   name="caps1_output_tile")
caps1_output_tiled = tf.tile(caps1_output_tile, [1, 1, caps2_n_caps, 1, 1],
                             name="caps1_output_tiled")


caps2_predicted = tf.matmul(W_tiled, caps1_output_tiled,
                            name="caps2_predicted")


raw_weights = tf.zeros([batch_size, caps1_n_caps, caps2_n_caps, 1, 1],
                       dtype=np.float32, name="raw_weights")


routing_weights = tf.nn.softmax(raw_weights, dim=2, name="routing_weights")



weighted_predictions = tf.multiply(routing_weights, caps2_predicted,
                                   name="weighted_predictions")
weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, keep_dims=True,
                             name="weighted_sum")

caps2_output_round_1 = squash(weighted_sum, axis=-2,
                              name="caps2_output_round_1")




caps2_output_round_1_tiled = tf.tile(
    caps2_output_round_1, [1, caps1_n_caps, 1, 1, 1],
    name="caps2_output_round_1_tiled")

agreement = tf.matmul(caps2_predicted, caps2_output_round_1_tiled,
                      transpose_a=True, name="agreement")

raw_weights_round_2 = tf.add(raw_weights, agreement,
                             name="raw_weights_round_2")

routing_weights_round_2 = tf.nn.softmax(raw_weights_round_2,
                                        dim=2,
                                        name="routing_weights_round_2")
weighted_predictions_round_2 = tf.multiply(routing_weights_round_2,
                                           caps2_predicted,
                                           name="weighted_predictions_round_2")
weighted_sum_round_2 = tf.reduce_sum(weighted_predictions_round_2,
                                     axis=1, keep_dims=True,
                                     name="weighted_sum_round_2")
caps2_output_round_2 = squash(weighted_sum_round_2,
                              axis=-2,
                              name="caps2_output_round_2")

caps2_output =  caps2_output_round_2

def safe_norm(s, axis=-1, epsilon=1e-7, keep_dims=False, name=None):
    with tf.name_scope(name, default_name="safe_norm"):
        squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                     keep_dims=keep_dims)
        return tf.sqrt(squared_norm + epsilon)


y_proba = safe_norm(caps2_output, axis=-2, name="y_proba")

y_proba_argmax = tf.argmax(y_proba, axis=2, name="y_proba")


y_pred = tf.squeeze(y_proba_argmax, axis=[1, 2], name="y_pred")


y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

m_plus = 0.9
m_minus = 0.1
lambda_ = 0.4


T = tf.one_hot(y, depth=caps2_n_caps, name="T")

caps2_output_norm = safe_norm(caps2_output, axis=-2, keep_dims=True,
                              name="caps2_output_norm")

present_error_raw = tf.square(tf.maximum(0., m_plus - caps2_output_norm),
                              name="present_error_raw")
present_error = tf.reshape(present_error_raw, shape=(-1, 10),
                           name="present_error")


absent_error_raw = tf.square(tf.maximum(0., caps2_output_norm - m_minus),
                             name="absent_error_raw")
absent_error = tf.reshape(absent_error_raw, shape=(-1, 10),
                          name="absent_error")

L = tf.add(T * present_error, lambda_ * (1.0 - T) * absent_error,
           name="L")


margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name="margin_loss")


mask_with_labels = tf.placeholder_with_default(False, shape=(),
                                               name="mask_with_labels")

reconstruction_targets = tf.cond(mask_with_labels,  # condition
                                 lambda: y,  # if True
                                 lambda: y_pred,  # if False
                                 name="reconstruction_targets")

reconstruction_mask = tf.one_hot(reconstruction_targets,
                                 depth=caps2_n_caps,
                                 name="reconstruction_mask")

reconstruction_mask_reshaped = tf.reshape(
    reconstruction_mask, [-1, 1, caps2_n_caps, 1, 1],
    name="reconstruction_mask_reshaped")


caps2_output_masked = tf.multiply(
    caps2_output, reconstruction_mask_reshaped,
    name="caps2_output_masked")

decoder_input = tf.reshape(caps2_output_masked,
                           [-1, caps2_n_caps * caps2_n_dims],
                           name="decoder_input")


n_hidden1 = 512
n_hidden2 = 1024
n_output = 28 * 28

with tf.name_scope("decoder"):
    hidden1 = tf.layers.dense(decoder_input, n_hidden1,
                              activation=tf.nn.relu,
                              name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2,
                              activation=tf.nn.relu,
                              name="hidden2")
    decoder_output = tf.layers.dense(hidden2, n_output,
                                     activation=tf.nn.sigmoid,
                                     name="decoder_output")


X_flat = tf.reshape(X, [-1, n_output], name="X_flat")
squared_difference = tf.square(X_flat - decoder_output,
                               name="squared_difference")
reconstruction_loss = tf.reduce_mean(squared_difference,
                                     name="reconstruction_loss")


alpha = 0.0005

loss = tf.add(margin_loss, alpha * reconstruction_loss, name="loss")


correct = tf.equal(y, y_pred, name="correct")
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

optimizer = tf.train.AdamOptimizer()
training_op = optimizer.minimize(loss, name="training_op")

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 1f
batch_size = 50
restore_checkpoint = False

n_iterations_per_epoch = mnist.train.num_examples // batch_size
n_iterations_validation = mnist.validation.num_examples // batch_size
best_loss_val = np.infty
checkpoint_path = "./my_capsule_network"

with tf.Session() as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init.run() 
    '''4 db tab/space'''

    for epoch in range(n_epochs):
        for iteration in range(1, n_iterations_per_epoch + 1):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            # Run the training operation and measure the loss:
            _, loss_train = sess.run(
                [training_op, loss],
                feed_dict={X: X_batch.reshape([-1, 28, 28, 1]),
                           y: y_batch,
                           mask_with_labels: True})
            print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                iteration, n_iterations_per_epoch,
                iteration * 100 / n_iterations_per_epoch,
                loss_train),
                end="")

        # At the end of each epoch,
        # measure the validation loss and accuracy:
        loss_vals = []
        acc_vals = []
        x=range(100)
        for iteration in range(1, n_iterations_validation + 1):
            X_batch, y_batch = mnist.validation.next_batch(batch_size)
            loss_val, acc_val = sess.run(
                [loss, accuracy],
                feed_dict={X: X_batch.reshape([-1, 28, 28, 1]),
                           y: y_batch})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)

            print("\rEvaluating!!! the model: {}/{} ({:.1f}%)".format(
                iteration, n_iterations_validation,
                iteration * 100 / n_iterations_validation),
                end=" " * 10)
        #plt.plot(x, acc_vals, label="acc_vals")
        #plt.plot(x, loss_vals, label="loss_vals")
        loss_val = np.mean(loss_vals)
        acc_val = np.mean(acc_vals)
        print("\rEpoch: {}  Val accuracy: {:.4f}%  Loss: {:.6f}{}".format(
            epoch + 1, acc_val * 100, loss_val,
            " (improved)" if loss_val < best_loss_val else ""))

        # And save the model if it improved:
        if loss_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = loss_val


n_iterations_test = mnist.test.num_examples // batch_size

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)

    loss_tests = []
    acc_tests = []
    for iteration in range(1, n_iterations_test + 1):
        X_batch, y_batch = mnist.test.next_batch(batch_size)
        loss_test, acc_test = sess.run(
            [loss, accuracy],
            feed_dict={X: X_batch.reshape([-1, 28, 28, 1]),
                       y: y_batch})
        loss_tests.append(loss_test)
        acc_tests.append(acc_test)
        print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
            iteration, n_iterations_test,
            iteration * 100 / n_iterations_test),
            end=" " * 10)
    loss_test = np.mean(loss_tests)
    acc_test = np.mean(acc_tests)
    print("\rFinal test accuracy: {:.4f}%  Loss: {:.6f}".format(
        acc_test * 100, loss_test))

In [0]:
n_samples = 10

sample_images = mnist.test.images[:n_samples].reshape([-1, 28, 28, 1])


In [0]:

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    #tf.initialize_all_variables().run()
    caps2_output_value, decoder_output_value, y_pred_value = sess.run(
        [caps2_output, decoder_output, y_pred],
        feed_dict={X: sample_images,
                   y: np.array([], dtype=np.int64)})hi

In [0]:
sample_images = sample_images.reshape(-1, 28, 28)
reconstructions = decoder_output_value.reshape([-1, 28, 28])

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    plt.imshow(sample_images[index], cmap="binary")
    plt.title("Címke:" + str(mnist.test.labels[index]))
    plt.axis("off")

plt.show()

plt.figure(figsize=(n_samples * 2, 3))
for index in range(n_samples):
    plt.subplot(1, n_samples, index + 1)
    plt.title("Rekonstruált:" + str(y_pred_value[index]))
    plt.imshow(reconstructions[index], cmap="binary")
    plt.axis("off")

plt.show()

In [0]:
def tweak_pose_parameters(output_vectors, min=-50, max=50, n_steps=11):
    steps = np.linspace(min, max, n_steps)  # -0.25, -0.15, ..., +0.25
    pose_parameters = np.arange(caps2_n_dims)  # 0, 1, ..., 15
    tweaks = np.zeros([caps2_n_dims, n_steps, 1, 1, 1, caps2_n_dims, 1])
    tweaks[pose_parameters, :, 0, 0, 0, pose_parameters, 0] = steps
    output_vectors_expanded = output_vectors[np.newaxis, np.newaxis]
    return tweaks + output_vectors_expanded

In [0]:

n_steps = 11

tweaked_vectors = tweak_pose_parameters(caps2_output_value, n_steps=n_steps)
tweaked_vectors_reshaped = tweaked_vectors.reshape(
    [-1, 1, caps2_n_caps, caps2_n_dims, 1])

tweak_labels = np.tile(mnist.test.labels[:n_samples], caps2_n_dims * n_steps)

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    decoder_output_value = sess.run(
        decoder_output,
        feed_dict={caps2_output: tweaked_vectors_reshaped,
                   mask_with_labels: True,
                   y: tweak_labels})

In [0]:
tweak_reconstructions = decoder_output_value.reshape(
    [caps2_n_dims, n_steps, n_samples, 28, 28])

for dim in range(3):
    print("Tweaking output dimension #{}".format(dim))
    plt.figure(figsize=(n_steps / 1.2, n_samples / 1.5))
    for row in range(n_samples):
        for col in range(n_steps):
            plt.subplot(n_samples, n_steps, row * n_steps + col + 1)
            plt.imshow(tweak_reconstructions[dim, col, row], cmap="binary")
            plt.axis("off")
    plt.show()
#tf.reset_default_graph()
#del conv1_params
#del conv2_params

# Ezek a kódok lényegtelenek

In [0]:
%reset
from keras import backend as K


K.get_session().close()
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))
print("jupy")

In [0]:
import sys
sys.path.append('local_modules')

In [0]:
# Now we can import our new module and call our function.
import demo_module
demo_module.SomeFunction(1)

In [0]:
#!rm -r *
#!find . ! -name 'datalab' -type d -exec rm -d {} +
!dir
!free -g
#!cat /proc/meminfo

cd tmp